# Clone Repo & Install Libraries

In [ ]:
from google.colab import userdata

GITHUB_PAT = userdata.get('github_pat')

github_username = 'westlund4228'
github_repo_name = 'ai502_term_project'

!git clone https://github.com/{github_username}/{github_repo_name}.git
%cd ai502_term_project

Cloning into 'amc3'...
remote: Enumerating objects: 168, done.
remote: Counting objects: 100% (168/168), done.
remote: Compressing objects: 100% (117/117), done.
remote: Total 168 (delta 95), reused 122 (delta 49), pack-reused 0 (from 0)
Receiving objects: 100% (168/168), 62.09 KiB | 512.00 KiB/s, done.
Resolving deltas: 100% (95/95), done.
/content/amc3


In [ ]:
!pip install tensorboardX torch-pruning

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.1/64.1 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 127.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 97.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 78.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 43.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 109.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-c

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sys, os
import zipfile
import time

def zip_folder(folder_path, zip_path, excluded=None):
    if not os.path.isdir(folder_path):
        print(f"Error: Folder not found - {folder_path}")
        return

    # Check folder size
    total_size = sum(
        os.path.getsize(os.path.join(root, file))
        for root, _, files in os.walk(folder_path)
        for file in files
        if os.path.isfile(os.path.join(root, file))
    )
    if total_size <= 1024:
        print(f"Warning: Small folder size detected - {folder_path} (≤1KB)")

    if os.path.exists(zip_path):
        raise FileExistsError(f"Zip file already exists - {zip_path}")

    excluded = excluded or []
    excluded = set(os.path.normpath(e) for e in excluded)

    with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk(folder_path):
            # Filter out excluded items
            dirs[:] = [d for d in dirs
                      if os.path.normpath(os.path.join(root, d)) not in excluded]

            for file in files:
                full_path = os.path.join(root, file)
                rel_path = os.path.relpath(full_path, folder_path)

                if os.path.normpath(rel_path) not in excluded:
                    zipf.write(full_path, arcname=rel_path)

    print(f"Success: Compressed {folder_path} to {zip_path}")


def get_sizes(folder_path):
    total_size = 0
    file_sizes = {}

    if not os.path.exists(folder_path):
        print(f"Error: Folder '{folder_path}' not found.")
        return

    print(f"File sizes in '{folder_path}':")
    for root, _, files in os.walk(folder_path):
        for file in files:
            file_path = os.path.join(root, file)
            try:
                size_bytes = os.path.getsize(file_path)
                total_size += size_bytes
                file_sizes[file_path] = size_bytes
                print(f"  {file_path}: {format_bytes(size_bytes)}")
            except OSError as e:
                print(f"  Error accessing file '{file_path}': {e}")

    print("\n---")
    print(f"Total size of '{folder_path}': {format_bytes(total_size)}")

def format_bytes(bytes_size):
    if bytes_size < 1024:
        return f"{bytes_size} Bytes"
    elif bytes_size < 1024**2:
        return f"{bytes_size / 1024:.2f} KB"
    elif bytes_size < 1024**3:
        return f"{bytes_size / (1024**2):.2f} MB"
    else:
        return f"{bytes_size / (1024**3):.2f} GB"


# MobileNetV1

## Download Weights

In [ ]:
!python scripts/download_mobilenet_weights.py

Model saved to models/state_dict/mobilenetv1_cifar.pth.tar


## AMC Baseline

In [ ]:
!python amc_search_no_sensitivity.py \
    --job=train \
    --model=mobilenet_cifar \
    --dataset=cifar10 \
    --preserve_ratio=0.5 \
    --acc_metric=acc1 \
    --data_bsize 128 \
    --n_worker 2 \
    --train_episode=200 \
    --reward=acc_flops_reward \
    --data_root=./data \
    --ckpt_path=./models/state_dict/mobilenetv1_cifar.pth.tar \
    --seed=9999 \
    --split_seed=2025 # Use split_seed=2025

=> Building model..
=> Loading checkpoint from ./models/state_dict/mobilenetv1_cifar.pth.tar...
=> Preparing data: cifar10...
100% 170M/170M [00:13<00:00, 12.6MB/s]
=> Building model..
=> Loading checkpoint from ./models/state_dict/mobilenetv1_cifar.pth.tar...
=> Preparing data: cifar10...
=> shape of embedding (n_layer * n_dim): (9, 10)
=> original acc: 86.040%
=> original weight size: 0.8118 M param
=> original FLOPs: 15.1685 M
=> Saving logs to ./logs/mobilenet_cifar_amc_r0.5_ep200_search-run1
=> Output path: ./logs/mobilenet_cifar_amc_r0.5_ep200_search-run1...
** Actual replay buffer size: 900
 New best reward: -7.9181, acc: 50.8200, compress: 0.6476
 New best policy: [0.9000162887906149, 0.6212741278231493, 1.0, 0.7946774141029013, 0.9246297821241971, 1.0, 0.41936359483482594, 1.0, 0.8583141046753631]
 Channels after pruning: [461, 319, 511, 204, 237, 127, 54, 63, 28]
#0: episode_reward:-7.9181 acc: 50.8200, ratio: 0.6476
New best clamped policy: [0.9000162887906149, 0.62127412782

In [ ]:
!python amc_search.py \
    --job=export \
    --model=mobilenet_cifar \
    --dataset=cifar10 \
    --acc_metric=acc1 \
    --data_bsize 128 \
    --data_root=./data \
    --reward=acc_flops_reward \
    --ckpt_path=./models/state_dict/mobilenetv1_cifar.pth.tar \
    --policy_path=/content/amc3/checkpoints/mobilenet_amc_r0.5_ep200_search/best_policy.txt \
    --export_path=./checkpoints/mobilenet_amc_r0.5_ep200_search/mobilenet_amc_r0.5_ep200_search.pt \
    --seed=9999 \
    --split_seed=2025 # Use split_seed=2025

=> Building model..
=> Loading checkpoint from ./models/state_dict/mobilenetv1_cifar.pth.tar...
=> Preparing data: cifar10...
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 12, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
Identified sensitive layers: ['features.0.0', 'features.1.0', 'features.2.0', 'features.3.0', 'features.4.0', 'features.5.0', 'features.6.0', 'features.7.0']
=> Building model..
=> Loading checkpoint from ./models/state_dict/mobilenetv1_cifar.pth.tar...
=> Preparing data: cifar10...
=> shape of embedding (n_layer * n_dim): (9, 10)
=> original acc: 86.040%
=> original weight size: 0.8118 M param
=> original FLOPs: 

In [ ]:
!python amc_fine_tune.py \
    --model=mobilenet_cifar \
    --dataset=cifar10 \
    --batch_size=128 \
    --n_worker=2 \
    --n_epoch=20 \
    --lr=0.005 \
    --lr_type=cos \
    --wd=4e-5 \
    --split_seed=2025 \
    --data_root=./data \
    --ckpt_path=./checkpoints/mobilenet_amc_r0.5_ep200_search/mobilenet_amc_r0.5_ep200_search.pt

=> Preparing data..
=> Preparing data: cifar10...
=> Building model..
=> Loading checkpoint from ./checkpoints/mobilenet_amc_r0.5_ep200_search/mobilenet_amc_r0.5_ep200_search.pt...
=> Model Parameter: 0.802 M, FLOPs: 14.716M
=> Building model..
=> Loading checkpoint from ./checkpoints/mobilenet_amc_r0.5_ep200_search/mobilenet_amc_r0.5_ep200_search.pt...
=> Resuming from checkpoint..
Using SGD...
weight decay  = 4e-05
=> Start training...
Training mobilenet_cifar on cifar10...
=> Saving logs to ./logs/mobilenet_cifar_finetune-run10
=> lr: 0.005

Epoch: 0
 [=======================================>]  Step: 149ms | Tot: 23s162ms | Loss: 0.487 | Acc1: 82.876% | Acc5: 99.304% 352/352 
 [=======================================>]  Step: 53ms | Tot: 623ms | Loss: 0.412 | Acc1: 85.880% | Acc5: 99.420% 40/40 
Current best acc: 85.88
=> Saving checkpoint to ./logs/mobilenet_cifar_finetune-run10/ckpt.pth.tar
=> lr: 0.0049692208514878445

Epoch: 1
 [=======================================>]  Step: 6

In [ ]:
zip_folder('/content/amc3/logs/mobilenet_cifar_finetune-run10', 'mobilenet_amc_r0.5_ep200_ft.zip', excluded=['ckpt.pth.tar'])

Success: Compressed /content/amc3/logs/mobilenet_cifar_finetune-run10 to mobilenet_amc_r0.5_ep200_ft.zip


## AMC, rbound=0.8

In [ ]:
!python amc_search_no_sensitivity.py \
    --job=train \
    --model=mobilenet_cifar \
    --dataset=cifar10 \
    --preserve_ratio=0.5 \
    --acc_metric=acc1 \
    --rbound=0.8 \
    --data_bsize 128 \
    --n_worker 2 \
    --train_episode=300 \
    --reward=acc_flops_reward \
    --data_root=./data \
    --init_delta=1.0 \
    --delta_decay=0.99 \
    --ckpt_path=./models/state_dict/mobilenetv1_cifar.pth.tar \
    --seed=10000 \
    --split_seed=2025

=> Building model..
=> Loading checkpoint from ./models/state_dict/mobilenetv1_cifar.pth.tar...
=> Preparing data: cifar10...
=> Building model..
=> Loading checkpoint from ./models/state_dict/mobilenetv1_cifar.pth.tar...
=> Preparing data: cifar10...
=> shape of embedding (n_layer * n_dim): (9, 10)
=> original acc: 86.040%
=> original weight size: 0.8118 M param
=> original FLOPs: 15.1685 M
=> Saving logs to ./logs/mobilenet_cifar_amc_r0.5_ep300_search-run9
=> Output path: ./logs/mobilenet_cifar_amc_r0.5_ep300_search-run9...
** Actual replay buffer size: 900
 New best reward: -10.2873, acc: 35.3600, compress: 0.5380
 New best policy: [0.8, 0.6212741278231493, 0.8, 0.7946774141029013, 0.8, 0.8, 0.41936359483482594, 0.8, 0.8]
 Channels after pruning: [410, 319, 410, 204, 205, 103, 54, 52, 26]
#0: episode_reward:-10.2873 acc: 35.3600, ratio: 0.5380
New best clamped policy: [0.9000162887906149, 0.6212741278231493, 1.0, 0.7946774141029013, 0.9246297821241971, 1.0, 0.41936359483482594, 1.0,

In [ ]:
zip_folder('/content/amc3/logs/mobilenet_cifar_amc_r0.5_ep300_search-run9', 'mobilenet_amc_r0.5_rbound0.8_ep300_search.zip')

In [ ]:
!python amc_search.py \
    --job=export \
    --model=mobilenet_cifar \
    --dataset=cifar10 \
    --acc_metric=acc1 \
    --data_bsize 128 \
    --data_root=./data \
    --reward=acc_flops_reward \
    --ckpt_path=./models/state_dict/mobilenetv1_cifar.pth.tar \
    --policy_path=/content/amc3/checkpoints/mobilenet_amc_r0.5_rbound0.8_ep300_search/best_policy.txt \
    --export_path=./checkpoints/mobilenet_amc_r0.5_rbound0.8_ep300_search/mobilenet_amc_r0.5_rbound0.8_ep300_search.pt \
    --seed=9999 \
    --split_seed=2025 # Use split_seed=2025

=> Building model..
=> Loading checkpoint from ./models/state_dict/mobilenetv1_cifar.pth.tar...
=> Preparing data: cifar10...
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 12, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
Identified sensitive layers: ['features.0.0', 'features.1.0', 'features.2.0', 'features.3.0', 'features.4.0', 'features.5.0', 'features.6.0', 'features.7.0']
=> Building model..
=> Loading checkpoint from ./models/state_dict/mobilenetv1_cifar.pth.tar...
=> Preparing data: cifar10...
=> shape of embedding (n_layer * n_dim): (9, 10)
=> original acc: 86.040%
=> original weight size: 0.8118 M param
=> original FLOPs: 

In [ ]:
!python amc_fine_tune.py \
    --model=mobilenet_cifar \
    --dataset=cifar10 \
    --batch_size=128 \
    --n_worker=2 \
    --n_epoch=20 \
    --lr=0.005 \
    --lr_type=cos \
    --wd=4e-5 \
    --split_seed=2025 \
    --data_root=./data \
    --ckpt_path=./checkpoints/mobilenet_amc_r0.5_rbound0.8_ep300_search/mobilenet_amc_r0.5_rbound0.8_ep300_search.pt

=> Preparing data..
=> Preparing data: cifar10...
=> Building model..
=> Loading checkpoint from ./checkpoints/mobilenet_amc_r0.5_rbound0.8_ep300_search/mobilenet_amc_r0.5_rbound0.8_ep300_search.pt...
=> Model Parameter: 0.528 M, FLOPs: 10.019M
=> Building model..
=> Loading checkpoint from ./checkpoints/mobilenet_amc_r0.5_rbound0.8_ep300_search/mobilenet_amc_r0.5_rbound0.8_ep300_search.pt...
=> Resuming from checkpoint..
Using SGD...
weight decay  = 4e-05
=> Start training...
Training mobilenet_cifar on cifar10...
=> Saving logs to ./logs/mobilenet_cifar_finetune-run11
=> lr: 0.005

Epoch: 0
 [=======================================>]  Step: 193ms | Tot: 23s200ms | Loss: 0.557 | Acc1: 80.640% | Acc5: 99.004% 352/352 
 [=======================================>]  Step: 55ms | Tot: 651ms | Loss: 0.451 | Acc1: 84.500% | Acc5: 99.360% 40/40 
Current best acc: 84.5
=> Saving checkpoint to ./logs/mobilenet_cifar_finetune-run11/ckpt.pth.tar
=> lr: 0.0049692208514878445

Epoch: 1
 [===========

In [ ]:
zip_folder('/content/amc3/logs/mobilenet_cifar_finetune-run11', 'mobilenet_amc_r0.5_rbound0.8_ep300_ft.zip', excluded=['ckpt.pth.tar'])

Success: Compressed /content/amc3/logs/mobilenet_cifar_finetune-run11 to mobilenet_amc_r0.5_rbound0.8_ep300_ft.zip


## Sensitivity

In [ ]:
!python amc_search.py \
    --job=train \
    --model=mobilenet_cifar \
    --dataset=cifar10 \
    --preserve_ratio=0.5 \
    --acc_metric=acc1 \
    --data_bsize 128 \
    --n_worker 2 \
    --train_episode=200 \
    --reward=acc_flops_reward \
    --data_root=./data \
    --ckpt_path=./models/state_dict/mobilenetv1_cifar.pth.tar \
    --seed=9999 \
    --split_seed=2025

=> Building model..
=> Loading checkpoint from ./models/state_dict/mobilenetv1_cifar.pth.tar...
=> Preparing data: cifar10...
Identified sensitive layers: ['features.0.0', 'features.1.0', 'features.2.0', 'features.3.0', 'features.4.0', 'features.5.0', 'features.6.0', 'features.7.0']
=> Building model..
=> Loading checkpoint from ./models/state_dict/mobilenetv1_cifar.pth.tar...
=> Preparing data: cifar10...
=> shape of embedding (n_layer * n_dim): (9, 10)
=> original acc: 86.040%
=> original weight size: 0.8118 M param
=> original FLOPs: 15.1685 M
Sensitive layer indices: [1, 3, 5, 7, 9, 11, 13, 15]
=> Saving logs to ./logs/mobilenet_cifar_sensitivity_r0.5_ep200_search-run2
=> Output path: ./logs/mobilenet_cifar_sensitivity_r0.5_ep200_search-run2...
** Actual replay buffer size: 900
 New best reward: -6.0333, acc: 63.0600, compress: 0.8171
 New best policy: [0.9000162887906149, 0.9, 1.0, 0.9, 0.9246297821241971, 1.0, 0.41936359483482594, 1.0, 0.8583141046753631]
 Channels after pruning:

In [ ]:
zip_folder('/content/amc3/logs/mobilenet_cifar_sensitivity_r0.5_ep200_search-run2', 'mobilenet_cifar_sensitivity_r0.5_ep200.zip')

Folder '/content/amc3/logs/mobilenet_cifar_sensitivity_r0.5_ep200_search-run2' successfully zipped to 'mobilenet_cifar_sensitivity_r0.5_ep200.zip'.


In [ ]:
!python amc_search.py \
    --job=export \
    --model=mobilenet_cifar \
    --dataset=cifar10 \
    --acc_metric=acc1 \
    --data_bsize 128 \
    --data_root=./data \
    --reward=acc_flops_reward \
    --ckpt_path=./models/state_dict/mobilenetv1_cifar.pth.tar \
    --policy_path=/content/amc3/checkpoints/mobilenet_cifar_sensitivity_r0.5_ep200_search/best_policy.txt \
    --export_path=./checkpoints/mobilenet_cifar_sensitivity_r0.5_ep200_search/mobilenet_cifar_sensitivity_r0.5_ep200_search.pt \
    --seed=9999 \
    --split_seed=2025 # Use split_seed=2025

=> Building model..
=> Loading checkpoint from ./models/state_dict/mobilenetv1_cifar.pth.tar...
=> Preparing data: cifar10...
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 12, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
Identified sensitive layers: ['features.0.0', 'features.1.0', 'features.2.0', 'features.3.0', 'features.4.0', 'features.5.0', 'features.6.0', 'features.7.0']
=> Building model..
=> Loading checkpoint from ./models/state_dict/mobilenetv1_cifar.pth.tar...
=> Preparing data: cifar10...
=> shape of embedding (n_layer * n_dim): (9, 10)
=> original acc: 86.040%
=> original weight size: 0.8118 M param
=> original FLOPs: 

In [ ]:
!python amc_fine_tune.py \
    --model=mobilenet_cifar \
    --dataset=cifar10 \
    --batch_size=128 \
    --n_worker=2 \
    --n_epoch=20 \
    --lr=0.005 \
    --lr_type=cos \
    --wd=4e-5 \
    --split_seed=2025 \
    --data_root=./data \
    --ckpt_path=./checkpoints/mobilenet_cifar_sensitivity_r0.5_ep200_search/mobilenet_cifar_sensitivity_r0.5_ep200_search.pt

=> Preparing data..
=> Preparing data: cifar10...
=> Building model..
=> Loading checkpoint from ./checkpoints/mobilenet_cifar_sensitivity_r0.5_ep200_search/mobilenet_cifar_sensitivity_r0.5_ep200_search.pt...
=> Model Parameter: 0.782 M, FLOPs: 14.397M
=> Building model..
=> Loading checkpoint from ./checkpoints/mobilenet_cifar_sensitivity_r0.5_ep200_search/mobilenet_cifar_sensitivity_r0.5_ep200_search.pt...
=> Resuming from checkpoint..
Using SGD...
weight decay  = 4e-05
=> Start training...
Training mobilenet_cifar on cifar10...
=> Saving logs to ./logs/mobilenet_cifar_finetune-run12
=> lr: 0.005

Epoch: 0
 [=======================================>]  Step: 148ms | Tot: 23s442ms | Loss: 0.493 | Acc1: 82.820% | Acc5: 99.249% 352/352 
 [=======================================>]  Step: 54ms | Tot: 623ms | Loss: 0.417 | Acc1: 85.280% | Acc5: 99.440% 40/40 
Current best acc: 85.28
=> Saving checkpoint to ./logs/mobilenet_cifar_finetune-run12/ckpt.pth.tar
=> lr: 0.0049692208514878445

Epoch

In [ ]:
zip_folder('/content/amc3/logs/mobilenet_cifar_finetune-run12', 'mobilenet_sensitivty_r0.5_ep200_ft.zip', excluded=['ckpt.pth.tar'])

Success: Compressed /content/amc3/logs/mobilenet_cifar_finetune-run12 to mobilenet_sensitivty_r0.5_ep200_ft.zip


## Sensitivity, rbound=0.8

In [ ]:
!python amc_search.py \
    --job=train \
    --model=mobilenet_cifar \
    --dataset=cifar10 \
    --preserve_ratio=0.5 \
    --acc_metric=acc1 \
    --rbound=0.8 \
    --data_bsize 128 \
    --n_worker 2 \
    --train_episode=300 \
    --reward=acc_flops_reward \
    --data_root=./data \
    --init_delta=1.0 \
    --delta_decay=0.99 \
    --ckpt_path=./models/state_dict/mobilenetv1_cifar.pth.tar \
    --seed=10000 \
    --split_seed=2025

=> Building model..
=> Loading checkpoint from ./models/state_dict/mobilenetv1_cifar.pth.tar...
=> Preparing data: cifar10...
Identified sensitive layers: ['conv1.0', 'features.0.0', 'features.1.0', 'features.2.0', 'features.3.0', 'features.4.0', 'features.5.0', 'features.6.0', 'features.7.0']
=> Building model..
=> Loading checkpoint from ./models/state_dict/mobilenetv1_cifar.pth.tar...
=> Preparing data: cifar10...
=> shape of embedding (n_layer * n_dim): (9, 10)
=> original acc: 86.040%
=> original weight size: 0.8118 M param
=> original FLOPs: 15.1685 M
Sensitive layer indices: [0, 1, 3, 5, 7, 9, 11, 13, 15]
=> Saving logs to ./logs/mobilenet_cifar_sensitivity_r0.5_rbound0.8_ep300_search-run3
=> Output path: ./logs/mobilenet_cifar_sensitivity_r0.5_rbound0.8_ep300_search-run3...
** Actual replay buffer size: 900
 New best reward: -8.8686, acc: 44.7600, compress: 0.6188
 New best policy: [0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.41936359483482594, 0.8, 0.8]
 Channels after pruning: [410, 410,

In [ ]:
zip_folder('/content/amc3/logs/mobilenet_cifar_sensitivity_r0.5_rbound0.8_ep300_search-run3', 'mobilenet_cifar_sensitivity_r0.5_rbound0.8_ep300_search.zip')

Error: The zip file 'mobilenet_cifar_sensitivity_r0.5_rbound0.8_ep300_search.zip' already exists. Please choose a different name or delete the existing file.


In [ ]:
!python amc_search.py \
    --job=export \
    --model=mobilenet_cifar \
    --dataset=cifar10 \
    --acc_metric=acc1 \
    --data_bsize 128 \
    --data_root=./data \
    --reward=acc_flops_reward \
    --ckpt_path=./models/state_dict/mobilenetv1_cifar.pth.tar \
    --policy_path=./checkpoints/mobilenet_cifar_sensitivity_r0.5_rbound0.8_ep300_search/best_policy.txt \
    --export_path=./checkpoints/mobilenet_cifar_sensitivity_r0.5_rbound0.8_ep300_search/mobilenet_cifar_sensitivity_r0.5_rbound0.8_ep300_search.pt \
    --seed=9999 \
    --split_seed=2025 # Use split_seed=2025

In [ ]:
!python amc_fine_tune.py \
    --model=mobilenet_cifar \
    --dataset=cifar10 \
    --batch_size=128 \
    --n_worker=2 \
    --n_epoch=20 \
    --lr=0.005 \
    --lr_type=cos \
    --wd=4e-5 \
    --split_seed=2025 \
    --data_root=./data \
    --ckpt_path=./checkpoints/mobilenet_cifar_sensitivity_r0.5_rbound0.8_ep300_search/mobilenet_cifar_sensitivity_r0.5_rbound0.8_ep300_search.pt

In [ ]:
zip_folder('/content/amc3/logs/mobilenet_cifar_finetune-run', 'mobilenet_sensitivity_r0.5_rbound0.8_ep300_ft.zip', excluded=['ckpt.pth.tar'])

# ResNet

## Download Weights

In [ ]:
!python scripts/download_resnet_weights.py

Model saved to models/state_dict/resnet_cifar.pth.tar


## AMC Baseline

In [ ]:
!python amc_search_no_sensitivity.py \
    --job=train \
    --model=resnet_cifar \
    --dataset=cifar10 \
    --preserve_ratio=0.5 \
    --acc_metric=acc1 \
    --data_bsize 128 \
    --n_worker 2 \
    --train_episode=200 \
    --reward=acc_flops_reward \
    --data_root=./data \
    --ckpt_path=./models/state_dict/resnet_cifar.pth.tar \
    --seed=9999 \
    --split_seed=2025 # Use split_seed=2025

=> Building model..
=> Loading checkpoint from ./models/state_dict/resnet_cifar.pth.tar...
=> Preparing data: cifar10...
=> Building model..
=> Loading checkpoint from ./models/state_dict/resnet_cifar.pth.tar...
=> Preparing data: cifar10...
=> shape of embedding (n_layer * n_dim): (11, 10)
=> original acc: 91.180%
=> original weight size: 11.1740 M param
=> original FLOPs: 140.8481 M
=> Saving logs to ./logs/resnet_cifar_sensitivity_r0.5_ep200_search-run1
=> Output path: ./logs/resnet_cifar_sensitivity_r0.5_ep200_search-run1...
** Actual replay buffer size: 1100
 New best reward: -4.4119, acc: 76.1000, compress: 0.7382
 New best policy: [0.9000162887906149, 0.6212741278231493, 1.0, 0.7946774141029013, 0.9246297821241971, 1.0, 0.41936359483482594, 1.0, 0.8583141046753631, 0.8962211148428955, 0.875450813354181]
 Channels after pruning: [461, 160, 127, 51, 60, 127, 54, 255, 220, 459, 449]
#0: episode_reward:-4.4119 acc: 76.1000, ratio: 0.7382
New best clamped policy: [0.9000162887906149,

In [ ]:
zip_folder('/content/amc3/logs/resnet_cifar_amc_r0.5_ep200_search-run1', 'resnet_amc_r0.5_ep200_search.zip')

Folder '/content/amc3/logs/resnet_cifar_amc_r0.5_ep200_search-run1' successfully zipped to 'resnet_amc_r0.5_ep200_search.zip'.


In [ ]:
!python amc_search_no_sensitivity.py \
    --job=export \
    --model=resnet_cifar \
    --dataset=cifar10 \
    --acc_metric=acc1 \
    --data_bsize 128 \
    --data_root=./data \
    --reward=acc_flops_reward \
    --ckpt_path=./models/state_dict/resnet_cifar.pth.tar \
    --policy_path=./checkpoints/resnet_amc_r0.5_ep200_search/best_policy.txt \
    --export_path=./checkpoints/resnet_amc_r0.5_ep200_search/resnet_amc_r0.5_ep200_search.pt \
    --seed=9999 \
    --split_seed=2025 # Use split_seed=2025

=> Building model..
=> Loading checkpoint from ./models/state_dict/resnet_cifar.pth.tar...
=> Preparing data: cifar10...
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 12, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
=> Building model..
=> Loading checkpoint from ./models/state_dict/resnet_cifar.pth.tar...
=> Preparing data: cifar10...
=> shape of embedding (n_layer * n_dim): (11, 10)
=> original acc: 91.180%
=> original weight size: 11.1740 M param
=> original FLOPs: 140.8481 M
=> Original model channels: [512, 256, 128, 64, 64, 128, 128, 256, 256, 512, 512]
=> Pruning with ratios: [0.9997167452889055, 0.9946738186056527, 0.994642

In [ ]:
!python amc_fine_tune.py \
    --model=resnet_cifar \
    --dataset=cifar10 \
    --batch_size=128 \
    --n_worker=2 \
    --n_epoch=20 \
    --lr=0.005 \
    --lr_type=cos \
    --wd=4e-5 \
    --split_seed=2025 \
    --data_root=./data \
    --ckpt_path=./checkpoints/resnet_amc_r0.5_ep200_search/resnet_amc_r0.5_ep200_search.pt

=> Preparing data..
=> Preparing data: cifar10...
=> Building model..
=> Loading checkpoint from ./checkpoints/resnet_amc_r0.5_ep200_search/resnet_amc_r0.5_ep200_search.pt...
=> Model Parameter: 9.091 M, FLOPs: 130.237M
=> Building model..
=> Loading checkpoint from ./checkpoints/resnet_amc_r0.5_ep200_search/resnet_amc_r0.5_ep200_search.pt...
=> Resuming from checkpoint..
Using SGD...
weight decay  = 4e-05
=> Start training...
Training resnet_cifar on cifar10...
=> Saving logs to ./logs/resnet_cifar_finetune-run4
=> lr: 0.005

Epoch: 0
 [=======================================>]  Step: 435ms | Tot: 23s676ms | Loss: 0.290 | Acc1: 90.178% | Acc5: 99.767% 352/352 
 [=======================================>]  Step: 137ms | Tot: 704ms | Loss: 0.288 | Acc1: 90.520% | Acc5: 99.640% 40/40 
Current best acc: 90.52
=> Saving checkpoint to ./logs/resnet_cifar_finetune-run4/ckpt.pth.tar
=> lr: 0.0049692208514878445

Epoch: 1
 [=======================================>]  Step: 13ms | Tot: 23s3ms | L

In [ ]:
zip_folder('/content/amc3/logs/resnet_cifar_finetune-run4', 'resnet_amc_r0.5_ep200_ft.zip', excluded=['ckpt.pth.tar'])

Success: Compressed /content/amc3/logs/resnet_cifar_finetune-run4 to resnet_amc_r0.5_ep200_ft.zip


In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

checkpoint = torch.load('/content/amc3/logs/resnet_cifar_finetune-run1/ckpt.best.pth.tar', map_location=device)
print("Loaded keys:", checkpoint.keys())

Loaded keys: dict_keys(['epoch', 'model', 'dataset', 'state_dict', 'acc', 'optimizer'])


## AMC, rbound=0.8

In [ ]:
!python amc_search_no_sensitivity.py \
    --job=train \
    --model=resnet_cifar \
    --dataset=cifar10 \
    --preserve_ratio=0.5 \
    --acc_metric=acc1 \
    --rbound=0.8 \
    --data_bsize 128 \
    --n_worker 2 \
    --train_episode=300 \
    --reward=acc_flops_reward \
    --data_root=./data \
    --init_delta=1.0 \
    --delta_decay=0.99 \
    --ckpt_path=./models/state_dict/resnet_cifar.pth.tar \
    --seed=10000 \
    --split_seed=2025

=> Building model..
=> Loading checkpoint from ./models/state_dict/resnet_cifar.pth.tar...
=> Preparing data: cifar10...
=> Building model..
=> Loading checkpoint from ./models/state_dict/resnet_cifar.pth.tar...
=> Preparing data: cifar10...
=> shape of embedding (n_layer * n_dim): (11, 10)
=> original acc: 91.180%
=> original weight size: 11.1740 M param
=> original FLOPs: 140.8481 M
=> Saving logs to ./logs/resnet_cifar_sensitivity_r0.5_rbound0.8_ep300_search-run4
=> Output path: ./logs/resnet_cifar_sensitivity_r0.5_rbound0.8_ep300_search-run4...
** Actual replay buffer size: 1100
 New best reward: -9.2875, acc: 49.2200, compress: 0.6228
 New best policy: [0.8, 0.6212741278231493, 0.8, 0.7946774141029013, 0.8, 0.8, 0.41936359483482594, 0.8, 0.8, 0.8, 0.8]
 Channels after pruning: [410, 160, 103, 51, 52, 103, 54, 205, 205, 410, 410]
#0: episode_reward:-9.2875 acc: 49.2200, ratio: 0.6228
New best clamped policy: [0.9000162887906149, 0.6212741278231493, 1.0, 0.7946774141029013, 0.924629

In [ ]:
zip_folder('/content/amc3/logs/resnet_cifar_amc_r0.5_rbound0.8_ep300_search-run4', 'resnet_cifar_amc_r0.5_rbound0.8_ep300_search.zip')

Folder '/content/amc3/logs/resnet_cifar_amc_r0.5_rbound0.8_ep300_search-run4' successfully zipped to 'resnet_cifar_amc_r0.5_rbound0.8_ep300_search.zip'.


In [ ]:
!python amc_search_no_sensitivity.py \
    --job=export \
    --model=resnet_cifar \
    --dataset=cifar10 \
    --acc_metric=acc1 \
    --data_bsize 128 \
    --data_root=./data \
    --reward=acc_flops_reward \
    --ckpt_path=./models/state_dict/resnet_cifar.pth.tar \
    --policy_path=/content/amc3/checkpoints/resnet_amc_r0.5_rbound0.8_ep300_search/best_policy.txt \
    --export_path=./checkpoints/resnet_amc_r0.5_rbound0.8_ep300_search/resnet_amc_r0.5_rbound0.8_ep300_search.pt \
    --seed=9999 \
    --split_seed=2025 # Use split_seed=2025

=> Building model..
=> Loading checkpoint from ./models/state_dict/resnet_cifar.pth.tar...
=> Preparing data: cifar10...
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 12, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
=> Building model..
=> Loading checkpoint from ./models/state_dict/resnet_cifar.pth.tar...
=> Preparing data: cifar10...
=> shape of embedding (n_layer * n_dim): (11, 10)
=> original acc: 91.180%
=> original weight size: 11.1740 M param
=> original FLOPs: 140.8481 M
=> Original model channels: [512, 256, 128, 64, 64, 128, 128, 256, 256, 512, 512]
=> Pruning with ratios: [0.9702728674442148, 0.9325206708821945, 0.967503

In [ ]:
!python amc_fine_tune.py \
    --model=resnet_cifar \
    --dataset=cifar10 \
    --batch_size=128 \
    --n_worker=2 \
    --n_epoch=20 \
    --lr=0.005 \
    --lr_type=cos \
    --wd=4e-5 \
    --split_seed=2025 \
    --data_root=./data \
    --ckpt_path=/content/amc3/checkpoints/resnet_amc_r0.5_rbound0.8_ep300_search/resnet_amc_r0.5_rbound0.8_ep300_search.pt

=> Preparing data..
=> Preparing data: cifar10...
=> Building model..
=> Loading checkpoint from /content/amc3/checkpoints/resnet_amc_r0.5_rbound0.8_ep300_search/resnet_amc_r0.5_rbound0.8_ep300_search.pt...
=> Model Parameter: 6.560 M, FLOPs: 112.829M
=> Building model..
=> Loading checkpoint from /content/amc3/checkpoints/resnet_amc_r0.5_rbound0.8_ep300_search/resnet_amc_r0.5_rbound0.8_ep300_search.pt...
=> Resuming from checkpoint..
Using SGD...
weight decay  = 4e-05
=> Start training...
Training resnet_cifar on cifar10...
=> Saving logs to ./logs/resnet_cifar_finetune-run5
=> lr: 0.005

Epoch: 0
 [=======================================>]  Step: 396ms | Tot: 23s169ms | Loss: 0.307 | Acc1: 89.420% | Acc5: 99.722% 352/352 
 [=======================================>]  Step: 152ms | Tot: 762ms | Loss: 0.294 | Acc1: 90.660% | Acc5: 99.620% 40/40 
Current best acc: 90.66
=> Saving checkpoint to ./logs/resnet_cifar_finetune-run5/ckpt.pth.tar
=> lr: 0.0049692208514878445

Epoch: 1
 [=======

In [ ]:
zip_folder('/content/amc3/logs/resnet_cifar_finetune-run5', 'resnet_amc_r0.5_rbound0.8_ep300_ft.zip', excluded=['ckpt.pth.tar'])

Success: Compressed /content/amc3/logs/resnet_cifar_finetune-run5 to resnet_amc_r0.5_rbound0.8_ep300_ft.zip


## Sensitivity

In [ ]:
!python amc_search.py \
    --job=train \
    --model=resnet_cifar \
    --dataset=cifar10 \
    --preserve_ratio=0.5 \
    --acc_metric=acc1 \
    --data_bsize 128 \
    --n_worker 2 \
    --train_episode=200 \
    --reward=acc_flops_reward \
    --data_root=./data \
    --ckpt_path=./models/state_dict/resnet_cifar.pth.tar \
    --seed=9999 \
    --split_seed=2025

=> Building model..
=> Loading checkpoint from ./models/state_dict/resnet_cifar.pth.tar...
=> Preparing data: cifar10...
Identified sensitive layers: ['conv1', 'layer1.0.conv1', 'layer1.0.conv2', 'layer1.1.conv1', 'layer1.1.conv2', 'layer2.0.conv2', 'layer2.1.conv1', 'layer2.1.conv2', 'layer3.1.conv1', 'layer3.1.conv2', 'layer4.0.conv1', 'layer4.0.conv2', 'layer4.0.downsample.0', 'layer4.1.conv1', 'layer4.1.conv2']
=> Building model..
=> Loading checkpoint from ./models/state_dict/resnet_cifar.pth.tar...
=> Preparing data: cifar10...
=> shape of embedding (n_layer * n_dim): (11, 10)
=> original acc: 91.180%
=> original weight size: 11.1740 M param
=> original FLOPs: 140.8481 M
Sensitive layer indices: [0, 1, 2, 3, 4, 6, 8, 9, 13, 14, 15, 16, 17, 18, 19]
=> Saving logs to ./logs/resnet_cifar_sensitivity_r0.5_ep200_search-run5
=> Output path: ./logs/resnet_cifar_sensitivity_r0.5_ep200_search-run5...
** Actual replay buffer size: 1100
 New best reward: -2.8170, acc: 84.8200, compress: 0.8

In [ ]:
zip_folder('./logs/resnet_cifar_sensitivity_r0.5_ep200_search-run5', 'resnet_cifar_sensitivity_r0.5_ep200_search.zip')

Error: The zip file 'resnet_cifar_sensitivity_r0.5_ep200_search.zip' already exists. Please choose a different name or delete the existing file.


In [ ]:
!python amc_search.py \
    --job=export \
    --model=resnet_cifar \
    --dataset=cifar10 \
    --acc_metric=acc1 \
    --data_bsize 128 \
    --data_root=./data \
    --reward=acc_flops_reward \
    --ckpt_path=./models/state_dict/resnet_cifar.pth.tar \
    --policy_path=/content/amc3/checkpoints/resnet_cifar_sensitivity_r0.5_ep200_search/best_policy.txt \
    --export_path=./checkpoints/resnet_cifar_sensitivity_r0.5_ep200_search/resnet_cifar_sensitivity_r0.5_ep200_search.pt \
    --seed=9999 \
    --split_seed=2025 # Use split_seed=2025

=> Building model..
=> Loading checkpoint from ./models/state_dict/resnet_cifar.pth.tar...
=> Preparing data: cifar10...
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 12, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
Identified sensitive layers: ['conv1', 'layer1.0.conv1', 'layer1.0.conv2', 'layer1.1.conv1', 'layer1.1.conv2', 'layer2.0.conv2', 'layer2.1.conv1', 'layer2.1.conv2', 'layer3.1.conv1', 'layer3.1.conv2', 'layer4.0.conv1', 'layer4.0.conv2', 'layer4.0.downsample.0', 'layer4.1.conv1', 'layer4.1.conv2']
=> Building model..
=> Loading checkpoint from ./models/state_dict/resnet_cifar.pth.tar...
=> Preparing data: cifar10...
=> 

In [ ]:
!python amc_fine_tune.py \
    --model=resnet_cifar \
    --dataset=cifar10 \
    --batch_size=128 \
    --n_worker=2 \
    --n_epoch=20 \
    --lr=0.005 \
    --lr_type=cos \
    --wd=4e-5 \
    --split_seed=2025 \
    --data_root=./data \
    --ckpt_path=./checkpoints/resnet_cifar_sensitivity_r0.5_ep200_search/resnet_cifar_sensitivity_r0.5_ep200_search.pt

=> Preparing data..
=> Preparing data: cifar10...
=> Building model..
=> Loading checkpoint from ./checkpoints/resnet_cifar_sensitivity_r0.5_ep200_search/resnet_cifar_sensitivity_r0.5_ep200_search.pt...
=> Model Parameter: 10.882 M, FLOPs: 137.304M
=> Building model..
=> Loading checkpoint from ./checkpoints/resnet_cifar_sensitivity_r0.5_ep200_search/resnet_cifar_sensitivity_r0.5_ep200_search.pt...
=> Resuming from checkpoint..
Using SGD...
weight decay  = 4e-05
=> Start training...
Training resnet_cifar on cifar10...
=> Saving logs to ./logs/resnet_cifar_finetune-run7
=> lr: 0.005

Epoch: 0
 [=======================================>]  Step: 388ms | Tot: 23s636ms | Loss: 0.286 | Acc1: 90.189% | Acc5: 99.762% 352/352 
 [=======================================>]  Step: 135ms | Tot: 688ms | Loss: 0.289 | Acc1: 89.980% | Acc5: 99.760% 40/40 
Current best acc: 89.98
=> Saving checkpoint to ./logs/resnet_cifar_finetune-run7/ckpt.pth.tar
=> lr: 0.0049692208514878445

Epoch: 1
 [==============

In [ ]:
zip_folder('/content/amc3/logs/resnet_cifar_finetune-run7', 'resnet_cifar_sensitivity_r0.5_ep200_ft.zip', excluded=['ckpt.pth.tar'])

Success: Compressed /content/amc3/logs/resnet_cifar_finetune-run7 to resnet_cifar_sensitivity_r0.5_ep200_ft.zip


## Sensitivity, rbound=0.8

In [ ]:
!python amc_search.py \
    --job=train \
    --model=resnet_cifar \
    --dataset=cifar10 \
    --preserve_ratio=0.5 \
    --acc_metric=acc1 \
    --rbound=0.8 \
    --data_bsize 128 \
    --n_worker 2 \
    --train_episode=300 \
    --reward=acc_flops_reward \
    --data_root=./data \
    --init_delta=1.0 \
    --delta_decay=0.99 \
    --ckpt_path=./models/state_dict/resnet_cifar.pth.tar \
    --seed=123456 \
    --split_seed=2025

=> Building model..
=> Loading checkpoint from ./models/state_dict/resnet_cifar.pth.tar...
=> Preparing data: cifar10...
Identified sensitive layers: ['conv1', 'layer1.0.conv1', 'layer1.0.conv2', 'layer1.1.conv1', 'layer1.1.conv2', 'layer2.1.conv2', 'layer3.1.conv1', 'layer3.1.conv2', 'layer4.0.conv1', 'layer4.0.conv2', 'layer4.0.downsample.0', 'layer4.1.conv1', 'layer4.1.conv2']
=> Building model..
=> Loading checkpoint from ./models/state_dict/resnet_cifar.pth.tar...
=> Preparing data: cifar10...
=> shape of embedding (n_layer * n_dim): (11, 10)
=> original acc: 91.180%
=> original weight size: 11.1740 M param
=> original FLOPs: 140.8481 M
Sensitive layer indices: [0, 1, 2, 3, 4, 9, 13, 14, 15, 16, 17, 18, 19]
=> Saving logs to ./logs/resnet_cifar_sensitivity_r0.5_rbound0.8_ep300_search-run6
=> Output path: ./logs/resnet_cifar_sensitivity_r0.5_rbound0.8_ep300_search-run6...
** Actual replay buffer size: 1100
 New best reward: -7.9146, acc: 56.8600, compress: 0.6590
 New best policy: 

In [ ]:
zip_folder('./logs/resnet_cifar_sensitivity_r0.5_rbound0.8_ep300_search-run6', 'resnet_cifar_sensitivity_r0.5_rbound0.8_ep300_search.zip')

Folder './logs/resnet_cifar_sensitivity_r0.5_rbound0.8_ep300_search-run6' successfully zipped to 'resnet_cifar_sensitivity_r0.5_rbound0.8_ep300_search.zip'.


In [ ]:
!python amc_search.py \
    --job=export \
    --model=resnet_cifar \
    --dataset=cifar10 \
    --acc_metric=acc1 \
    --data_bsize 128 \
    --data_root=./data \
    --reward=acc_flops_reward \
    --ckpt_path=./models/state_dict/resnet_cifar.pth.tar \
    --policy_path=/content/amc3/checkpoints/resnet_cifar_sensitivity_r0.5_rbound0.8_ep300_search/best_policy.txt \
    --export_path=./checkpoints/resnet_cifar_sensitivity_r0.5_rbound0.8_ep300_search/resnet_cifar_sensitivity_r0.5_rbound0.8_ep300_search.pt \
    --seed=9999 \
    --split_seed=2025 # Use split_seed=2025

=> Building model..
=> Loading checkpoint from ./models/state_dict/resnet_cifar.pth.tar...
=> Preparing data: cifar10...
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 12, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
Identified sensitive layers: ['conv1', 'layer1.0.conv1', 'layer1.0.conv2', 'layer1.1.conv1', 'layer1.1.conv2', 'layer2.0.conv2', 'layer2.1.conv1', 'layer2.1.conv2', 'layer3.1.conv1', 'layer3.1.conv2', 'layer4.0.conv1', 'layer4.0.conv2', 'layer4.0.downsample.0', 'layer4.1.conv1', 'layer4.1.conv2']
=> Building model..
=> Loading checkpoint from ./models/state_dict/resnet_cifar.pth.tar...
=> Preparing data: cifar10...
=> 

In [ ]:
!python amc_fine_tune.py \
    --model=resnet_cifar \
    --dataset=cifar10 \
    --batch_size=128 \
    --n_worker=2 \
    --n_epoch=20 \
    --lr=0.005 \
    --lr_type=cos \
    --wd=4e-5 \
    --split_seed=2025 \
    --data_root=./data \
    --ckpt_path=./checkpoints/resnet_cifar_sensitivity_r0.5_rbound0.8_ep300_search/resnet_cifar_sensitivity_r0.5_rbound0.8_ep300_search.pt

=> Preparing data..
=> Preparing data: cifar10...
=> Building model..
=> Loading checkpoint from ./checkpoints/resnet_cifar_sensitivity_r0.5_rbound0.8_ep300_search/resnet_cifar_sensitivity_r0.5_rbound0.8_ep300_search.pt...
=> Model Parameter: 5.927 M, FLOPs: 104.041M
=> Building model..
=> Loading checkpoint from ./checkpoints/resnet_cifar_sensitivity_r0.5_rbound0.8_ep300_search/resnet_cifar_sensitivity_r0.5_rbound0.8_ep300_search.pt...
=> Resuming from checkpoint..
Using SGD...
weight decay  = 4e-05
=> Start training...
Training resnet_cifar on cifar10...
=> Saving logs to ./logs/resnet_cifar_finetune-run8
=> lr: 0.005

Epoch: 0
 [=======================================>]  Step: 374ms | Tot: 23s703ms | Loss: 0.322 | Acc1: 88.960% | Acc5: 99.662% 352/352 
 [=======================================>]  Step: 136ms | Tot: 844ms | Loss: 0.311 | Acc1: 89.960% | Acc5: 99.600% 40/40 
Current best acc: 89.96
=> Saving checkpoint to ./logs/resnet_cifar_finetune-run8/ckpt.pth.tar
=> lr: 0.0049692

In [ ]:
zip_folder('/content/amc3/logs/resnet_cifar_finetune-run8', 'resnet_cifar_sensitivity_r0.5_rbound0.8_ep300_ft.zip', excluded=['ckpt.pth.tar'])

Success: Compressed /content/amc3/logs/resnet_cifar_finetune-run8 to resnet_cifar_sensitivity_r0.5_rbound0.8_ep300_ft.zip
